In [23]:
import numpy as np
import pandas as pd
import geopandas as gpd
import os
pd.set_option('display.max_columns', 50)

Link to open dataset which does not have lat-longs.

In [8]:
# test = pd.read_csv('https://data.austintexas.gov/api/views/7d8e-dm7r/rows.csv?accessType=DOWNLOAD')

In [4]:
# Ensuring that the pandas parser interprets dates as dates
dockless = pd.read_csv('https://s3.amazonaws.com/sameera-bucket-1/dockless_mobility/raw_data/Austin_Dockless_Vehicle_Trips.csv'\
                       ,infer_datetime_format=True,parse_dates=['Start Time','End Time'],encoding='ascii')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# Fixing the names of the columns
dockless.columns = dockless.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

In [12]:
dockless.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2746505 entries, 0 to 2746504
Data columns (total 20 columns):
id                        object
device_id                 object
vehicle_type              object
trip_duration             float64
trip_distance             float64
start_time                datetime64[ns]
end_time                  datetime64[ns]
modified_date             object
month                     float64
hour                      float64
day_of_week               float64
council_district_start    float64
council_district_end      float64
origin_cell_id            object
destination_cell_id       object
year                      float64
start_latitude            float64
start_longitude           float64
end_latitude              float64
end_longitude             float64
dtypes: datetime64[ns](2), float64(12), object(6)
memory usage: 419.1+ MB


In [20]:
dockless.origin_cell_id.value_counts()[:15]

014706           60248
014707           58180
014864           56408
014705           52553
015186           46612
015019           42919
014865           42406
014863           42273
014391           41524
015029           38675
014715           38111
OUT_OF_BOUNDS    37798
014873           37255
015021           34557
014549           32671
Name: origin_cell_id, dtype: int64

#### Origin cell id has 'Out_of_bounds' values in many cases so; looking into that

In [15]:
dockless[dockless.origin_cell_id=='OUT_OF_BOUNDS'][:10]

,id,device_id,vehicle_type,trip_duration,trip_distance,start_time,end_time,modified_date,month,hour,day_of_week,council_district_start,council_district_end,origin_cell_id,destination_cell_id,year,start_latitude,start_longitude,end_latitude,end_longitude
118,00034de2-79b4-44e6-b0a4-cf4a6515e51c,722ceae4-82f1-4f4f-93dd-d1de1030de3e,scooter,425.0,1299.0,2018-10-29 13:45:00,2018-10-29 14:00:00,12/12/2018 09:31:05 PM,10.0,13.0,1.0,0.0,0.0,OUT_OF_BOUNDS,OUT_OF_BOUNDS,2018.0,0.0,0.0,0.000,0.000
128,00039939-bea8-4a05-bccc-6da0b6d851d3,db16a992-42a0-45a2-857a-b7d0eebc755b,scooter,57.0,1.0,2018-10-29 11:45:00,2018-10-29 11:45:00,12/13/2018 03:49:28 PM,10.0,11.0,1.0,0.0,0.0,OUT_OF_BOUNDS,OUT_OF_BOUNDS,2018.0,0.0,0.0,0.000,0.000
267,00080fda-ef07-4c72-81bb-b331ac087a31,ad1e67ea-ae3b-4f55-b158-32d61c9198cd,scooter,63.0,0.0,2018-10-17 17:45:00,2018-10-17 17:45:00,12/13/2018 03:38:06 PM,10.0,17.0,3.0,0.0,9.0,OUT_OF_BOUNDS,014872,2018.0,NaN,NaN,30.286,-97.741
453,000e6942-3a96-4384-93af-97ec721c4cc0,8ea6275e-7122-48f2-89c8-b6d93f553204,scooter,408.0,1400.0,2018-11-30 14:00:00,2018-11-30 14:00:00,12/13/2018 03:54:39 PM,11.0,14.0,5.0,0.0,9.0,OUT_OF_BOUNDS,015357,2018.0,NaN,NaN,30.315,-97.734
588,00126ad5-74ad-4073-98f3-95bc158de3a4,b6e3766e-2166-4ce6-808e-182846f4523f,scooter,582.0,1000.0,2018-10-10 12:45:00,2018-10-10 13:00:00,12/13/2018 03:39:37 PM,10.0,12.0,3.0,0.0,10.0,OUT_OF_BOUNDS,014882,2018.0,NaN,NaN,30.309,-97.742
616,001313bb-d8a3-4ba7-ba7b-5c79cd2a00d1,add78e92-55ff-421c-a0f0-b2fe66e41f6b,scooter,556.0,580.0,2018-09-30 14:30:00,2018-09-30 14:45:00,12/13/2018 03:52:53 PM,9.0,14.0,0.0,0.0,0.0,OUT_OF_BOUNDS,OUT_OF_BOUNDS,2018.0,NaN,NaN,NaN,NaN
630,001377ba-2c67-4384-8f0b-edfaaeb8e65a,1d816fcf-d573-48cd-ae68-e6cf364e2432,scooter,5287.0,10000.0,2018-10-14 14:45:00,2018-10-14 16:00:00,12/12/2018 09:11:41 PM,10.0,14.0,0.0,0.0,5.0,OUT_OF_BOUNDS,013442,2018.0,NaN,NaN,30.255,-97.762
677,0015a86e-446c-4be0-9529-ff9a683cc859,e0ae258e-92d0-433e-bba6-4d8dfef63abf,scooter,923.0,800.0,2018-12-03 08:00:00,2018-12-03 08:15:00,12/13/2018 03:49:05 PM,12.0,8.0,1.0,0.0,9.0,OUT_OF_BOUNDS,015029,2018.0,NaN,NaN,30.283,-97.738
894,001c5303-4826-4696-bbb4-7399973abf75,94571d86-0acf-4c42-9796-f0e509e15a80,scooter,240.0,200.0,2018-11-07 19:00:00,2018-11-07 19:00:00,12/12/2018 09:31:05 PM,11.0,19.0,3.0,0.0,3.0,OUT_OF_BOUNDS,015807,2018.0,NaN,NaN,30.263,-97.727
974,001e8570-19c2-4b1f-8e92-f9bb4a6049a7,c97c4f15-95a8-40a2-8ac9-814aa4600923,scooter,1315.0,2300.0,2018-11-11 13:30:00,2018-11-11 13:45:00,12/13/2018 03:39:56 PM,11.0,13.0,0.0,0.0,9.0,OUT_OF_BOUNDS,015023,2018.0,NaN,NaN,30.271,-97.739


#### It turns out that these are the rows for which start locations or end locations are missing. So, these rows will be useless for any spatial analysis. So, it would be a good idea to drop them.

In [17]:
dockless.shape

(2746505, 20)

In [40]:
# Removing all rows where either origin cell id or destination cell id is 'OUT_OF_BOUNDS'
dockless = dockless.loc[(dockless.origin_cell_id!='OUT_OF_BOUNDS') & (dockless.destination_cell_id!='OUT_OF_BOUNDS')]

In [41]:
# Keeeping track of how many rows we dropped
dockless.shape

(2695253, 20)

#### There might still be other null, zero and Nan values in the spatial columns so, those rows also need to be dropped.

In [68]:
dockless.drop(dockless[dockless['start_latitude'].isnull() | dockless['start_latitude']==0 | dockless['start_latitude'].isna()].index,inplace=True)
dockless.drop(dockless[dockless['start_longitude'].isnull() | dockless['start_longitude']==0 | dockless['start_longitude'].isna()].index,inplace=True)
dockless.drop(dockless[dockless['end_latitude'].isnull() | dockless['end_latitude']==0 | dockless['end_latitude'].isna() ].index,inplace=True)
dockless.drop(dockless[dockless['end_longitude'].isnull() | dockless['end_longitude']==0  | dockless['end_longitude'].isna()].index,inplace=True)

In [69]:
# Keeeping track of how many rows we dropped
dockless.shape

(2688724, 20)

In [73]:
dockless.vehicle_type.unique()

array(['scooter', 'bicycle'], dtype=object)

### Dropping bicycles as we are looking to analyse the patterns of scooters

In [75]:
dockless = dockless[dockless.vehicle_type=='scooter']

In [76]:
# Keeeping track of how many rows we dropped
dockless.shape

(2612879, 20)

In [77]:
# Saving the processed csv
dockless.to_csv('Dockless_data_Austin.csv',index=False)

### Getting the shape file for the Austin Council Districts

In [46]:
url = "https://data.austintexas.gov/api/geospatial/hdpc-ysmz?method=export&format=Shapefile"
print('Downloading shapefile...')
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
print("Done")
if not os.path.isdir('council_districts'): z.extractall(path='council_districts/') # extract to folder

Done


In [47]:
filename_council = [f for f in os.listdir('council_districts/') for ending in ['dbf', 'prj', 'shp', 'shx'] if f.endswith(ending)]
print(filename_council)

['geo_export_a9cbcd82-61df-4e90-b67f-b7649d24806a.shx', 'geo_export_a9cbcd82-61df-4e90-b67f-b7649d24806a.shp', 'geo_export_a9cbcd82-61df-4e90-b67f-b7649d24806a.dbf', 'geo_export_a9cbcd82-61df-4e90-b67f-b7649d24806a.prj']


In [37]:
dbf, prj, shp, shx = [filename for filename in filename_council]
council_districts = gpd.read_file('council_districts/'+shp)

In [15]:
council_districts.head()

,council_di,created_by,date_creat,time_creat,modified_b,date_modif,time_modif,objectid,shape_area,shape_leng,single_mem,geometry
0,http://www.austintexas.gov/department/district-3,meekss,2018-06-14,11:00:25.000,lockwoodg,2016-10-18,13:41:59.000,1.0,5.027911e+08,153982.266647,400378072.0,"(POLYGON ((-97.680452610835 30.267642217703, -..."
1,http://www.austintexas.gov/department/district-4,meekss,2018-06-14,11:00:26.000,lockwoodg,2016-10-18,13:42:05.000,3.0,3.308754e+08,105092.713667,400378084.0,"POLYGON ((-97.70834462297 30.392488241669, -97..."
2,http://www.austintexas.gov/department/district-2,meekss,2018-06-14,11:00:26.000,lockwoodg,2016-10-18,13:41:55.000,2.0,1.290120e+09,689873.424949,400378076.0,"(POLYGON ((-97.718093991782 30.184857710517, -..."
3,http://www.austintexas.gov/department/district-9,meekss,2018-06-14,11:00:26.000,lockwoodg,2016-10-18,13:42:32.000,4.0,3.482170e+08,135780.245550,400378086.0,"POLYGON ((-97.731632625359 30.317929225339, -9..."
4,http://www.austintexas.gov/department/district-5,meekss,2018-06-14,11:00:25.000,meekss,2018-03-30,16:36:17.000,8.0,6.719064e+08,343441.587183,400378068.0,"(POLYGON ((-97.79137132583099 30.093084953282,..."


### Getting the shape file for the Austin Census Tracts 

In [7]:
import requests, zipfile, io

census_url = 'https://data.austintexas.gov/download/e228-ig6a/application%2Fzip'
r = requests.get(census_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall('CT_Austin/')
# filename = [y for y in sorted(z.namelist()) if y.endswith('shp')] 

In [20]:
filenames = [f for f in os.listdir('CT_Austin/') for ending in ['dbf', 'prj', 'shp', 'shx'] if f.endswith(ending)]

In [21]:
dbf, prj, shp, shx = [filename for filename in filenames]
ct = gpd.read_file('CT_Austin/'+shp)

In [22]:
ct.head()

,TRACTCE10,SUM_totpop,SUM_white,SUM_black,SUM_hispan,SUM_asian,SUM_other,SUM_multi,SUM_hu_tot,SUM_hu_occ,...,SUM_AGE651,SUM_AGE_65,SUM_AGE75_,SUM_AGE80_,SUM_AGE_85,SUM_FAMSWC,SUM_POP_IN,SUM_FAMILI,SUM_NONFAM,geometry
0,000101,3611.0,3130.0,39.0,276.0,90.0,9.0,67.0,2076.0,1815.0,...,38.0,151.0,79.0,103.0,303.0,354.0,77.0,805.0,1010.0,"POLYGON ((3110996.149334418 10094866.83662638,..."
1,000102,2552.0,2311.0,23.0,150.0,42.0,3.0,23.0,1153.0,1081.0,...,70.0,253.0,77.0,66.0,91.0,299.0,18.0,735.0,346.0,"POLYGON ((3108183.235349492 10097092.04712778,..."
2,000203,1546.0,1073.0,87.0,213.0,143.0,5.0,25.0,817.0,735.0,...,8.0,25.0,6.0,6.0,2.0,32.0,281.0,122.0,613.0,"POLYGON ((3117475.629412891 10088913.11716985,..."
3,000204,3009.0,2396.0,43.0,323.0,161.0,8.0,78.0,1750.0,1596.0,...,21.0,80.0,27.0,20.0,26.0,234.0,56.0,495.0,1101.0,"POLYGON ((3114760.420262146 10081981.02697965,..."
4,000205,3394.0,2333.0,110.0,676.0,162.0,24.0,89.0,2153.0,1996.0,...,35.0,95.0,30.0,28.0,21.0,212.0,0.0,557.0,1439.0,"POLYGON ((3118490.457187081 10090915.15399349,..."
